In [14]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn import linear_model
import numpy
import random
import gzip
import math

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
def assertFloat(x): # Checks that an answer is a float
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [17]:
f = gzip.open("young_adult_10000.json.gz")
dataset = []
for l in f:
    dataset.append(json.loads(l))

FileNotFoundError: [Errno 2] No such file or directory: 'young_adult_10000.json.gz'

In [12]:
len(dataset)

0

In [ ]:
answers = {} # Put your answers to each question in this dictionary

In [ ]:
dataset[0]

In [ ]:
### Question 1

In [18]:
def feature(datum):
    # your implementation
    feat = datum['review_text'].count('!')
    return [1] + [feat]

In [ ]:
X = [feature(d) for d in dataset]
Y = [d['rating'] for d in dataset]

In [ ]:
theta, residual, rank, s = numpy.linalg.lstsq(X, Y)

In [ ]:
answers['Q1'] = [theta0, theta1, mse]

In [ ]:
assertFloatList(answers['Q1'], 3) # Check the format of your answer (three floats)

In [ ]:
### Question 2

In [ ]:
def feature(datum):
    

In [ ]:
X = 
Y = 

In [ ]:
answers['Q2'] = [theta0, theta1, theta2, mse]

In [ ]:
assertFloatList(answers['Q2'], 4)

In [ ]:
### Question 3

In [ ]:
def feature(datum, deg):
    # feature for a specific polynomial degree

In [ ]:
answers['Q3'] = mses

In [ ]:
assertFloatList(answers['Q3'], 5)# List of length 5

In [ ]:
### Question 4

In [ ]:
answers['Q4'] = mses

In [ ]:
assertFloatList(answers['Q4'], 5)

In [ ]:
### Question 5

In [ ]:
answers['Q5'] = mae

In [ ]:
assertFloat(answers['Q5'])

In [ ]:
### Question 6

In [ ]:
f = open("beer_50000.json")
dataset = []
for l in f:
    if 'user/gender' in l:
        dataset.append(eval(l))

In [ ]:
len(dataset)

In [ ]:
X = 
y = 

In [ ]:
answers['Q6'] = [TP, TN, FP, FN, BER]

In [ ]:
assertFloatList(answers['Q6'], 5)

In [ ]:
### Question 7

In [ ]:
answers["Q7"] = [TP, TN, FP, FN, BER]

In [ ]:
assertFloatList(answers['Q7'], 5)

In [ ]:
### Question 8

In [ ]:
answers['Q8'] = precisionList

In [ ]:
assertFloatList(answers['Q8'], 5) #List of five floats

In [ ]:
f = open("answers_hw1.txt", 'w') # Write your answers to a file
f.write(str(answers) + '\n')
f.close()